### Prepare Fannie Mae Single Family mortgage dataset

**Intoduction**  
The initial population of mortgage loans in the primary dataset included Fannie Mae acquisitions between January 1, 2000 and March 31, 2012 with corresponding monthly performance data as of December 31, 2012.

The compressed dataset is about 50GB.

**Main steps**
- Download data from https://datadynamics.fanniemae.com/data-dynamics/#/reportMenu;category=HP
- Join datasets
- Create default flag

**Main references**
- Introduction: https://capitalmarkets.fanniemae.com/credit-risk-transfer/single-family-credit-risk-transfer/fannie-mae-single-family-loan-performance-data
- Data dictionary: https://capitalmarkets.fanniemae.com/media/20926/display
- Data preparation: https://github.com/ferrarisf50/Fannie-Mae-single-family-mortgage-loan-data/blob/master/Predict_default.ipynb


#### A. Import dataset

In [22]:
import os, glob, csv
from pyspark.sql import SparkSession
from tqdm import tqdm
import pandas as pd

from pyspark.sql.functions import col, when
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Initialize a Spark session and set options
spark = SparkSession.builder.appName("GetFannieMaeDataset").getOrCreate()
pd.set_option("display.max_columns", None)

In [2]:
headers = [
    "POOL_ID",
    "LOAN_ID",
    "ACT_PERIOD",
    "CHANNEL",
    "SELLER",
    "SERVICER",
    "MASTER_SERVICER",
    "ORIG_RATE",
    "CURR_RATE",
    "ORIG_UPB",
    "ISSUANCE_UPB",
    "CURRENT_UPB",
    "ORIG_TERM",
    "ORIG_DATE",
    "FIRST_PAY",
    "LOAN_AGE",
    "REM_MONTHS",
    "ADJ_REM_MONTHS",
    "MATR_DT",
    "OLTV",
    "OCLTV",
    "NUM_BO",
    "DTI",
    "CSCORE_B",
    "CSCORE_C",
    "FIRST_FLAG",
    "PURPOSE",
    "PROP",
    "NO_UNITS",
    "OCC_STAT",
    "STATE",
    "MSA",
    "ZIP",
    "MI_PCT",
    "PRODUCT",
    "PPMT_FLG",
    "IO",
    "FIRST_PAY_IO",
    "MNTHS_TO_AMTZ_IO",
    "DLQ_STATUS",
    "PMT_HISTORY",
    "MOD_FLAG",
    "MI_CANCEL_FLAG",
    "Zero_Bal_Code",
    "ZB_DTE",
    "LAST_UPB",
    "RPRCH_DTE",
    "CURR_SCHD_PRNCPL",
    "TOT_SCHD_PRNCPL",
    "UNSCHD_PRNCPL_CURR",
    "LAST_PAID_INSTALLMENT_DATE",
    "FORECLOSURE_DATE",
    "DISPOSITION_DATE",
    "FORECLOSURE_COSTS",
    "PROPERTY_PRESERVATION_AND_REPAIR_COSTS",
    "ASSET_RECOVERY_COSTS",
    "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS",
    "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY",
    "NET_SALES_PROCEEDS",
    "CREDIT_ENHANCEMENT_PROCEEDS",
    "REPURCHASES_MAKE_WHOLE_PROCEEDS",
    "OTHER_FORECLOSURE_PROCEEDS",
    "NON_INTEREST_BEARING_UPB",
    "PRINCIPAL_FORGIVENESS_AMOUNT",
    "ORIGINAL_LIST_START_DATE",
    "ORIGINAL_LIST_PRICE",
    "CURRENT_LIST_START_DATE",
    "CURRENT_LIST_PRICE",
    "ISSUE_SCOREB",
    "ISSUE_SCOREC",
    "CURR_SCOREB",
    "CURR_SCOREC",
    "MI_TYPE",
    "SERV_IND",
    "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT",
    "CUMULATIVE_MODIFICATION_LOSS_AMOUNT",
    "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS",
    "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS",
    "HOMEREADY_PROGRAM_INDICATOR",
    "FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT",
    "RELOCATION_MORTGAGE_INDICATOR",
    "ZERO_BALANCE_CODE_CHANGE_DATE",
    "LOAN_HOLDBACK_INDICATOR",
    "LOAN_HOLDBACK_EFFECTIVE_DATE",
    "DELINQUENT_ACCRUED_INTEREST",
    "PROPERTY_INSPECTION_WAIVER_INDICATOR",
    "HIGH_BALANCE_LOAN_INDICATOR",
    "ARM_5_YR_INDICATOR",
    "ARM_PRODUCT_TYPE",
    "MONTHS_UNTIL_FIRST_PAYMENT_RESET",
    "MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET",
    "INTEREST_RATE_CHANGE_DATE",
    "PAYMENT_CHANGE_DATE",
    "ARM_INDEX",
    "ARM_CAP_STRUCTURE",
    "INITIAL_INTEREST_RATE_CAP",
    "PERIODIC_INTEREST_RATE_CAP",
    "LIFETIME_INTEREST_RATE_CAP",
    "MARGIN",
    "BALLOON_INDICATOR",
    "PLAN_NUMBER",
    "FORBEARANCE_INDICATOR",
    "HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR",
    "DEAL_NAME",
    "RE_PROCS_FLAG",
    "ADR_TYPE",
    "ADR_COUNT",
    "ADR_UPB",
    # "PAYMENT_DEFERRAL_MOD_EVENT_FLAG",
    # "INTEREST_BEARING_UPB",
]

len(headers)

108

In [13]:
# Get list of CSV files
csv_files = glob.glob("C:\\projects\\risk_datasets\\data\\*.csv")

# Output path
output_path = "C:\\projects\\risk_datasets\\final_result.csv"

# Write the headers to the output file
with open(output_path, 'w', newline='') as f_out:
    writer = csv.writer(f_out, delimiter='|')
    writer.writerow(headers)

# Function to read a CSV file and append its content to the output file
def append_csv_to_output(file_path, output_path):
    with open(file_path, 'r') as f_in:
        reader = csv.reader(f_in, delimiter='|')
        with open(output_path, 'a', newline='') as f_out:
            writer = csv.writer(f_out, delimiter='|')
            for row in reader:
                writer.writerow(row)

# Loop through the CSV files and append them to the output CSV
for file in tqdm(csv_files, desc="Processing CSV files"):
    append_csv_to_output(file, output_path)

Processing CSV files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [09:15<00:00, 138.99s/it]


#### B. Get a sample from the dataset

In [23]:
# Read the first 10 rows of the CSV file
csv_path = "C:\\projects\\risk_datasets\\final_result.csv"
first_rows = pd.read_csv(csv_path, sep='|', nrows=10)

first_rows.head()

,POOL_ID,LOAN_ID,ACT_PERIOD,CHANNEL,SELLER,SERVICER,MASTER_SERVICER,ORIG_RATE,CURR_RATE,ORIG_UPB,ISSUANCE_UPB,CURRENT_UPB,ORIG_TERM,ORIG_DATE,FIRST_PAY,LOAN_AGE,REM_MONTHS,ADJ_REM_MONTHS,MATR_DT,OLTV,OCLTV,NUM_BO,DTI,CSCORE_B,CSCORE_C,FIRST_FLAG,PURPOSE,PROP,NO_UNITS,OCC_STAT,STATE,MSA,ZIP,MI_PCT,PRODUCT,PPMT_FLG,IO,FIRST_PAY_IO,MNTHS_TO_AMTZ_IO,DLQ_STATUS,PMT_HISTORY,MOD_FLAG,MI_CANCEL_FLAG,Zero_Bal_Code,ZB_DTE,LAST_UPB,RPRCH_DTE,CURR_SCHD_PRNCPL,TOT_SCHD_PRNCPL,UNSCHD_PRNCPL_CURR,LAST_PAID_INSTALLMENT_DATE,FORECLOSURE_DATE,DISPOSITION_DATE,FORECLOSURE_COSTS,PROPERTY_PRESERVATION_AND_REPAIR_COSTS,ASSET_RECOVERY_COSTS,MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS,ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY,NET_SALES_PROCEEDS,CREDIT_ENHANCEMENT_PROCEEDS,REPURCHASES_MAKE_WHOLE_PROCEEDS,OTHER_FORECLOSURE_PROCEEDS,NON_INTEREST_BEARING_UPB,PRINCIPAL_FORGIVENESS_AMOUNT,ORIGINAL_LIST_START_DATE,ORIGINAL_LIST_PRICE,CURRENT_LIST_START_DATE,CURRENT_LIST_PRICE,ISSUE_SCOREB,ISSUE_SCOREC,CURR_SCOREB,CURR_SCOREC,MI_TYPE,SERV_IND,CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT,CUMULATIVE_MODIFICATION_LOSS_AMOUNT,CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS,CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS,HOMEREADY_PROGRAM_INDICATOR,FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT,RELOCATION_MORTGAGE_INDICATOR,ZERO_BALANCE_CODE_CHANGE_DATE,LOAN_HOLDBACK_INDICATOR,LOAN_HOLDBACK_EFFECTIVE_DATE,DELINQUENT_ACCRUED_INTEREST,PROPERTY_INSPECTION_WAIVER_INDICATOR,HIGH_BALANCE_LOAN_INDICATOR,ARM_5_YR_INDICATOR,ARM_PRODUCT_TYPE,MONTHS_UNTIL_FIRST_PAYMENT_RESET,MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET,INTEREST_RATE_CHANGE_DATE,PAYMENT_CHANGE_DATE,ARM_INDEX,ARM_CAP_STRUCTURE,INITIAL_INTEREST_RATE_CAP,PERIODIC_INTEREST_RATE_CAP,LIFETIME_INTEREST_RATE_CAP,MARGIN,BALLOON_INDICATOR,PLAN_NUMBER,FORBEARANCE_INDICATOR,HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR,DEAL_NAME,RE_PROCS_FLAG,ADR_TYPE,ADR_COUNT,ADR_UPB
0,NaN,100000778481,112006,C,"Citimortgage, Inc.","Citimortgage Asset Management, Inc.",NaN,6.0,6.0,92000.0,NaN,0.0,360,102006,122006,0,360,360,112036,80,80,1,35,614,NaN,N,C,SF,1,P,SC,43900,293,NaN,FRM,N,N,NaN,NaN,0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,7,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN
1,NaN,100000778481,122006,C,"Citimortgage, Inc.","Citimortgage Asset Management, Inc.",NaN,6.0,6.0,92000.0,NaN,0.0,360,102006,122006,1,359,359,112036,80,80,1,35,614,NaN,N,C,SF,1,P,SC,43900,293,NaN,FRM,N,N,NaN,NaN,0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,7,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN
2,NaN,100000778481,12007,C,"Citimortgage, Inc.","Citimortgage Asset Management, Inc.",NaN,6.0,6.0,92000.0,NaN,0.0,360,102006,122006,2,358,358,112036,80,80,1,35,614,NaN,N,C,SF,1,P,SC,43900,293,NaN,FRM,N,N,NaN,NaN,0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,7,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN
3,NaN,100000778481,22007,C,"Citimortgage, Inc.","Citimortgage Asset Management, Inc.",NaN,6.0,6.0,92000.0,NaN,0.0,360,102006,122006,3,357,357,112036,80,80,1,35,614,NaN,N,C,SF,1,P,SC,43900,293,NaN,FRM,N,N,NaN,NaN,0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,7,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN
4,NaN,100000778481,32007,C,"Citimortgage, Inc.","Citimortgage Asset Management, Inc.",NaN,6.0,6.0,92000.0,NaN,0.0,360,102006,122006,4,356,355,112036,80,80,1,35,614,NaN,N,C,SF,1,P,SC,43900,293,NaN,FRM,N,N,NaN,NaN,0,NaN,N,N

In [17]:
# Import the complete dataset
csv_path = "C:\\projects\\risk_datasets\\final_result.csv"
data = spark.read.csv(csv_path, sep='|', header=True, inferSchema=True)

data.limit(10).show()

+-------+------------+----------+-------+------------------+--------------------+---------------+---------+---------+--------+------------+-----------+---------+---------+---------+--------+----------+--------------+-------+----+-----+------+---+--------+--------+----------+-------+----+--------+--------+-----+-----+---+------+-------+--------+---+------------+----------------+----------+-----------+--------+--------------+-------------+------+--------+---------+----------------+---------------+------------------+--------------------------+----------------+----------------+-----------------+--------------------------------------+--------------------+------------------------------------------+-------------------------------------+------------------+---------------------------+-------------------------------+--------------------------+------------------------+----------------------------+------------------------+-------------------+-----------------------+------------------+------------+

#### B. Append default flag

Zero_Bal_Code values:
- 01 = Prepaid or Matured
- 02 = Third Party Sale
- 03 = Short Sale06 = Repurchased
- 09 = Deed-in-Lieu; REO Disposition
- 15 = Non Performing Note Sale
- 16 = Reperforming Note Sale
- 96 = Removal (non-credit event)Applies to all CAS deals prior to and including 2015-C03:
- 97 = Delinquency (credit event due to D180)
- 98 = Other Credit Event

In [26]:
# Create a default flag
data = data.withColumn("is_first_credit_event",
                       (col("Zero_Bal_Code").isin(["03", "09"])) | 
                       ((col("DLQ_STATUS") < 999) & (col("DLQ_STATUS") >= 6)))

#### C. Review the final dataset

In [28]:
data.groupBy("Zero_Bal_Code").count().show()

+-------------+--------+
|Zero_Bal_Code|   count|
+-------------+--------+
|         NULL|41068945|
|            1|  535521|
|           16|    5200|
|            6|    1622|
|            3|    5370|
|           15|    1421|
|            9|   16471|
|            2|    1602|
+-------------+--------+



In [ ]:
data.groupBy("Default").count().show()